In [6]:
%load_ext autoreload
%pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/ocean/projects/asc170022p/mtragoza/lung-project'

In [7]:
import sys
sys.path.append('../param_search')
import param_search as ps

In [8]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT
nvidia-smi

data_root=../../data/{data_name}
data_root=$(realpath $data_root)

case_name=Case{case_id:d}Pack
case_dir=$data_root/$case_name

image_dir=$case_dir/NIFTI
mask_dir=$case_dir/TotalSegment
output_dir=$case_dir/CorrField

mkdir -p $output_dir

fixed_name=case{case_id:d}_T{fixed_phase:02d}
fixed_image=$image_dir/$fixed_name.nii.gz
fixed_mask=$mask_dir/$fixed_name/lung_combined_mask.nii.gz

moving_name=case{case_id:d}_T{moving_phase:02d}
moving_image=$image_dir/$moving_name.nii.gz

reg_name=case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}
output_path=$output_dir/$reg_name

cd ../../../Lung250M-4B
python -m corrfield -F $fixed_image -M $moving_image -m $fixed_mask -O $output_path

echo Done
'''
name_format = 'register_{data_name}_case{case_id:d}_T{moving_phase:02d}_T{fixed_phase:02d}'

In [9]:
param_space = ps.ParamSpace(
    data_name='Emory-4DCT',
    case_id=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    fixed_phase=[50],
    moving_phase=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
)

for p in param_space:
    print(name_format.format(**p))
    
print(len(param_space))

register_Emory-4DCT_case1_T00_T50
register_Emory-4DCT_case1_T10_T50
register_Emory-4DCT_case1_T20_T50
register_Emory-4DCT_case1_T30_T50
register_Emory-4DCT_case1_T40_T50
register_Emory-4DCT_case1_T50_T50
register_Emory-4DCT_case1_T60_T50
register_Emory-4DCT_case1_T70_T50
register_Emory-4DCT_case1_T80_T50
register_Emory-4DCT_case1_T90_T50
register_Emory-4DCT_case2_T00_T50
register_Emory-4DCT_case2_T10_T50
register_Emory-4DCT_case2_T20_T50
register_Emory-4DCT_case2_T30_T50
register_Emory-4DCT_case2_T40_T50
register_Emory-4DCT_case2_T50_T50
register_Emory-4DCT_case2_T60_T50
register_Emory-4DCT_case2_T70_T50
register_Emory-4DCT_case2_T80_T50
register_Emory-4DCT_case2_T90_T50
register_Emory-4DCT_case3_T00_T50
register_Emory-4DCT_case3_T10_T50
register_Emory-4DCT_case3_T20_T50
register_Emory-4DCT_case3_T30_T50
register_Emory-4DCT_case3_T40_T50
register_Emory-4DCT_case3_T50_T50
register_Emory-4DCT_case3_T60_T50
register_Emory-4DCT_case3_T70_T50
register_Emory-4DCT_case3_T80_T50
register_Emory

In [ ]:
jobs = ps.submit(template, name_format, param_space, work_dir='2023-01-15_register')
jobs.to_csv('2023-01-15_register.jobs')
jobs

In [10]:
import pandas as pd
jobs = pd.read_csv('2023-01-15_register.jobs', index_col=0)
status = ps.status(jobs)
status

,index,data_name,case_id,fixed_phase,moving_phase,job_name,partition,job_state,node_id,runtime,work_dir,array_idx,stdout,stderr
job_id,,,,,,,,,,,,,,
21681668,0,Emory-4DCT,1,50,0,register_Emory-4DCT_case1_T00_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681669,1,Emory-4DCT,1,50,10,register_Emory-4DCT_case1_T10_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681670,2,Emory-4DCT,1,50,20,register_Emory-4DCT_case1_T20_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681671,3,Emory-4DCT,1,50,30,register_Emory-4DCT_case1_T30_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681672,4,Emory-4DCT,1,50,40,register_Emory-4DCT_case1_T40_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v026.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21681763,95,Emory-4DCT,10,50,50,register_Emory-4DCT_case10_T50_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v031.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681764,96,Emory-4DCT,10,50,60,register_Emory-4DCT_case10_T60_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v031.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...
21681765,97,Emory-4DCT,10,50,70,register_Emory-4DCT_case10_T70_T50,GPU-shared,NaN,NaN,NaN,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,v031.ib.bridges2.psc.edu\n/ocean/projects/asc1...,/ocean/projects/asc170022p/mtragoza/mambaforge...


In [11]:
print(status.iloc[0].stderr)

/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/ocean/projects/asc170022p/mtragoza/Lung250M-4B/corrfield/mindssc.py:46: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400366987/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  F.conv3d(rpad(img), mshift1, dilation=delta) -
/ocean/projects/asc170022p/mtragoza/Lung250M-4B/corrfield/graphs.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /op

## Check output files

In [12]:
import numpy as np
import xarray as xr
import torch
import nibabel as nib
import hvplot.xarray

disp_file = 'data/Emory-4DCT/Case1Pack/CorrField/case1_T00_T50.nii.gz'
disp = torch.from_numpy(
    nib.load(disp_file).get_fdata().astype(np.float32)
)
disp.shape

torch.Size([1, 256, 256, 94, 3])

In [13]:
%autoreload
import data

ds = data.Emory4DCTDataset(data_root='data/Emory-4DCT', case_names=['Case1Pack'])
ds.load_niftis()
ds.load_masks(roi='lung_combined_mask')
ds.load_displacements(fixed_phase=50)
ds.load_landmarks()

Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T10.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T20.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T30.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T40.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T60.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T70.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T80.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T90.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T00/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T10/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T20/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T30/lung_combined_mask.nii.gz
Loading data/Emory-4DCT/Case1Pack/TotalSegment/case1_T40/lung_combined_mask.nii.gz
Loading data/Em

In [20]:
data.view_array(ds[0].array, groupby=['phase', 'y'], x='x', y='z', clim=(-1000, 1000))

:DynamicMap   [phase,y]
   :Image   [x,z]   (CT)

In [22]:
data.view_array(ds[0].mask, groupby=['phase', 'y'], x='x', y='z', clim=(0, 1))

:DynamicMap   [phase,y]
   :Image   [x,z]   (mask)

In [23]:
data.view_array(ds[0].disp, groupby=['phase', 'y', 'component'], x='x', y='z', clim=(-10, 10), cmap='seismic')

:DynamicMap   [phase,y,component]
   :Image   [x,z]   (displacement)

In [30]:
ds[0].landmarks[0].shape

(75, 3)

In [24]:
import torch
import torch.nn.functional as F
sys.path.append('../Lung250M-4B')
import corrfield

In [28]:
ds[0].disp.sel(phase=0)

<xarray.DataArray 'displacement' (x: 256, y: 256, z: 94, component: 3)>
array([[[[-0.19725657,  0.18855788,  3.72065806],
         [-0.19202898,  0.22886273,  3.72118616],
         [-0.18680154,  0.2691676 ,  3.72171426],
         ...,
         [-0.99159646,  1.33488357,  2.63766527],
         [-0.97847289,  1.33290517,  2.64906716],
         [-0.96534938,  1.33092666,  2.66046929]],

        [[-0.19068624,  0.17473309,  3.68711996],
         [-0.18529861,  0.21557614,  3.6876483 ],
         [-0.17991096,  0.25641915,  3.68817663],
         ...,
         [-0.98869127,  1.32499492,  2.62248778],
         [-0.97549385,  1.32283068,  2.63378382],
         [-0.96229696,  1.32066631,  2.64507985]],

        [[-0.18411592,  0.16090827,  3.65358114],
         [-0.17856809,  0.20228951,  3.65410995],
         [-0.17302063,  0.24367069,  3.65463829],
         ...,
...
         ...,
         [ 1.85563898, -2.40501404,  1.17633808],
         [ 1.88574946, -2.40881371,  1.20866549],
         [ 1.91586041, -2.41261339,  1.24099326]],

        [[ 3.06607032, -5.70032978, -2.49166203],
         [ 3.04808044, -5.6315341 , -2.43805313],
         [ 3.03009081, -5.56273794, -2.384444  ],
         ...,
         [ 1.85949504, -2.43069506,  1.18595457],
         [ 1.88910162, -2.43423462,  1.21795213],
         [ 1.91870844, -2.43777442,  1.24994993]],

        [[ 3.06481266, -5.72020769, -2.47239232],
         [ 3.04646468, -5.65178394, -2.41900229],
         [ 3.02811646, -5.5833602 , -2.36561227],
         ...,
         [ 1.86335087, -2.45637584,  1.19557106],
         [ 1.89245379, -2.45965552,  1.22723877],
         [ 1.92155671, -2.46293497,  1.25890672]]]])
Coordinates:
    phase      int64 0
  * x          (x) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * y          (y) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * z          (z) float64 0.0 2.5 5.0 7.5 10.0 ... 225.0 227.5 230.0 232.5
  * component  (component) <U1 'x' 'y' 'z'

In [48]:
nib.load('data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz').shape

(256, 256, 94)

In [78]:
device = 'cuda'

img_mov = torch.from_numpy(
    ds[0].array.sel(phase=0).values.astype(np.float32)
).unsqueeze(0).unsqueeze(0).to(device)

img_fix = torch.from_numpy(
    ds[0].array.sel(phase=50).values.astype(np.float32)
).unsqueeze(0).unsqueeze(0).to(device)

mask_fix = torch.from_numpy(
    ds[0].mask.sel(phase=50).values.astype(np.float32)
).unsqueeze(0).unsqueeze(0).to(device)

img_mov.shape, img_fix.shape

(torch.Size([1, 1, 256, 256, 94]), torch.Size([1, 1, 256, 256, 94]))

In [67]:
dense_flow_world = torch.from_numpy(ds[0].disp.sel(phase=0).values).unsqueeze(0).to(device)
dense_flow_world.shape

torch.Size([1, 256, 256, 94, 3])

In [68]:
_, _, D, H, W = img_fix.shape

dense_flow_pt = corrfield.utils.flow_pt(
    dense_flow_world.reshape(1, -1, 3), (D, H, W), align_corners=True
).reshape(1, D, H, W, 3)

In [69]:
img_mov_warped = F.grid_sample(
    img_mov,
    F.affine_grid(
        torch.eye(3, 4, dtype=img_mov.dtype, device=device).unsqueeze(0),
        (1, 1, D, H, W),
        align_corners=True
    ) + dense_flow_pt.to(img_mov.dtype),
    align_corners=True
)

In [77]:
data.view_array(
    xr.DataArray(
        img_mov_warped.squeeze(0).squeeze(0).detach().cpu().numpy(),
        dims=['x', 'y', 'z'],
        coords={
            'x': ds[0].array.x,
            'y': ds[0].array.y,
            'z': ds[0].array.z,
        }
    ),
    groupby=['y'],
    x='x',
    y='z',
    clim=(-1000, 1000)
)

:DynamicMap   [y]
   :Image   [x,z]   (value)

In [79]:
torch.norm((img_fix - img_mov) * mask_fix)

tensor(128641.0625, device='cuda:0')

In [80]:
torch.norm((img_fix - img_mov_warped) * mask_fix)

tensor(61094.1602, device='cuda:0')

In [86]:
data.view_array(
    xr.DataArray(
        torch.cat([
            img_mov, img_fix, img_mov_warped
        ], dim=0).squeeze(1).detach().cpu().numpy(),
        dims=['image', 'x', 'y', 'z'],
        coords={
            'image': ['moving', 'fixed', 'warped'],
            'x': ds[0].array.x,
            'y': ds[0].array.y,
            'z': ds[0].array.z,
        }
    ),
    groupby=['y', 'image'],
    x='x',
    y='z',
    clim=(-1000, 1000)
)

:DynamicMap   [y,image]
   :Image   [x,z]   (value)

In [93]:
kpts_mov_world = torch.from_numpy(ds[0].landmarks[0]).unsqueeze(0).to(device)
kpts_mov_world.shape

torch.Size([1, 75, 3])

In [88]:
kpts_fix_world = torch.from_numpy(ds[0].landmarks[5]).unsqueeze(0).to(device)
kpts_fix_world.shape

torch.Size([1, 75, 3])

In [201]:
kpts_fix_pt = corrfield.utils.kpts_pt(kpts_fix_world, (D, H, W), align_corners=True)
kpts_fix_pt.shape

torch.Size([1, 75, 3])

In [202]:
kpts_flow_pt = F.grid_sample(
    dense_flow_pt.permute(0, 4, 1, 2, 3),
    kpts_fix_pt.view(1, 1, 1, -1, 3),
    align_corners=True
).view(1, 3, -1).permute(0, 2, 1)
kpts_flow_pt.shape

torch.Size([1, 75, 3])

In [203]:
kpts_mov_corr_pt = kpts_fix_pt + kpts_flow_pt

In [204]:
kpts_mov_corr_world = corrfield.utils.kpts_world(kpts_mov_corr_pt, (D, H, W), align_corners=True)

In [205]:
torch.norm(kpts_mov_world - kpts_fix_world)

tensor(20.3224, device='cuda:0', dtype=torch.float64)

In [206]:
torch.norm(kpts_mov_world - kpts_mov_corr_world)

tensor(8.0271, device='cuda:0', dtype=torch.float64)